In [1]:
cd ..

/home/abhishekabhishek/git/UnsupGenModbyMPS


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pennylane as qml
from pennylane import numpy as pnp
import tensornetwork as tn

from MPScumulant import MPS_c
import mps_circuit_helpers as helpers
import mps_circuit
import metrics

%load_ext autoreload

In [3]:
# load and format the data
data = np.load('BStest/BSdata.npy')
data_states = data.reshape(-1, 16).astype(np.int8)

In [4]:
# load the trained MPS (with max_bond_dim = 4)
m = MPS_c(16, max_bond_dim=4)
m.loadMPS('BS-4-MPS')

# check the properties of the matrices in the MPS
for i in range(len(m.matrices)):
    tn_core = m.matrices[i]
    print(i, tn_core.shape)

0 (1, 2, 2)
1 (2, 2, 4)
2 (4, 2, 4)
3 (4, 2, 4)
4 (4, 2, 4)
5 (4, 2, 4)
6 (4, 2, 4)
7 (4, 2, 4)
8 (4, 2, 4)
9 (4, 2, 4)
10 (4, 2, 4)
11 (4, 2, 4)
12 (4, 2, 4)
13 (4, 2, 4)
14 (4, 2, 2)
15 (2, 2, 1)


In [17]:
type(m)

MPScumulant.MPS_c

Attempt to create a tensornetwork MPS using the above core tensors

In [6]:
tn_mps = tn.FiniteMPS(m.matrices, canonicalize=False)

In [7]:
tn_mps

In [10]:
tn_mps.bond_dimensions, tn_mps.physical_dimensions

([1, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 1],
 [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [11]:
tn_mps.check_canonical()

ValueError: FiniteMPS.center_positions is `None`. Cannot check canonical form.

In [14]:
# try to left canonicalize the MPS
tn_mps.center_position = len(m.matrices)-1
tn_mps.position(len(m.matrices)-1)

1.4142135623730951

In [15]:
tn_mps.check_canonical()

0.7090506896529838

In [16]:
tn_mps.canonicalize()

0.9999999999999999

Check if all the tensor are left isometries after left canonicalization

In [18]:
%autoreload
# check for isometries by reshaping along axis 0
helpers.are_isometries(tn_mps)

idx, core tensor shape, left isometry, right isometry
0 (1, 2, 2) True True
1 (2, 2, 4) True True
2 (4, 2, 4) True False
3 (4, 2, 4) True False
4 (4, 2, 4) True False
5 (4, 2, 4) True False
6 (4, 2, 4) True False
7 (4, 2, 4) True False
8 (4, 2, 4) True False
9 (4, 2, 4) True False
10 (4, 2, 4) True False
11 (4, 2, 4) True False
12 (4, 2, 4) True False
13 (4, 2, 4) True False
14 (4, 2, 2) True False
15 (2, 2, 1) True False


Ok. So looks like left-canonicalization using the tensornetwork methods was successful.